# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
df = df.replace(['OWN'],'1')
df = df.replace(['MORTGAGE'],'2')
df = df.replace(['RENT'],'3')
df = df.replace(['ANY'],'4')

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,2,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,2,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,3,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,2,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
df.isnull().mean() * 100

loan_amnt                     0.0
int_rate                      0.0
installment                   0.0
home_ownership                0.0
annual_inc                    0.0
verification_status           0.0
issue_d                       0.0
loan_status                   0.0
pymnt_plan                    0.0
dti                           0.0
delinq_2yrs                   0.0
inq_last_6mths                0.0
open_acc                      0.0
pub_rec                       0.0
revol_bal                     0.0
total_acc                     0.0
initial_list_status           0.0
out_prncp                     0.0
out_prncp_inv                 0.0
total_pymnt                   0.0
total_pymnt_inv               0.0
total_rec_prncp               0.0
total_rec_int                 0.0
total_rec_late_fee            0.0
recoveries                    0.0
collection_recovery_fee       0.0
last_pymnt_amnt               0.0
next_pymnt_d                  0.0
collections_12_mths_ex_med    0.0
policy_code   

In [7]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

label_encoder = LabelEncoder()

In [8]:
label_encoder.fit(df["issue_d"])

LabelEncoder()

In [9]:
list(label_encoder.classes_)

['Feb-2019', 'Jan-2019', 'Mar-2019']

In [10]:
verified_labels = {"Not Verified": 0,
     "Source Verified": 1,
     "Verified": 1
        
    }

risk_labels = {"low_risk": 0,
     "high_risk": 1 }

issue_labels = {
    "Mar-2019": 3,
     "Feb-2019": 2,
     "Jan-2019": 1           
               }

initial_labels = {"w": 0,
     "f": 1
                }

payment_labels = {
    "May-2019": 5,
    "Apr-2019": 4         
               }

In [11]:
df["verified_num"] = df["verification_status"].apply(lambda x: verified_labels[x])

In [12]:
df["risk_num"] = df["loan_status"].apply(lambda x: risk_labels[x])

In [13]:
df["issue_num"] = df["issue_d"].apply(lambda x: issue_labels[x])

In [14]:
df["initial_num"] = df["initial_list_status"].apply(lambda x: initial_labels[x])

In [15]:
df["next_pay_num"] = df["next_pymnt_d"].apply(lambda x: payment_labels[x])

In [16]:
df_2 = pd.get_dummies(df, columns=["application_type","hardship_flag","debt_settlement_flag","pymnt_plan"])
df_2.tail()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,dti,delinq_2yrs,...,verified_num,risk_num,issue_num,initial_num,next_pay_num,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,pymnt_plan_n
68812,10000.0,0.1502,346.76,3,26000.0,Source Verified,Jan-2019,low_risk,9.60,0.0,...,1,0,1,0,5,1,0,1,1,1
68813,12000.0,0.2727,368.37,3,63000.0,Not Verified,Jan-2019,low_risk,29.07,0.0,...,0,0,1,0,5,1,0,1,1,1
68814,5000.0,0.1992,185.62,2,52000.0,Source Verified,Jan-2019,low_risk,14.86,0.0,...,1,0,1,0,5,1,0,1,1,1
68815,40000.0,0.0646,1225.24,2,520000.0,Verified,Jan-2019,low_risk,9.96,0.0,...,1,0,1,1,5,1,0,1,1,1
68816,16000.0,0.1131,350.36,2,72000.0,Verified,Jan-2019,low_risk,7.02,2.0,...,1,0,1,0,5,1,0,1,1,1


In [17]:
df_2.drop(["verification_status","loan_status","issue_d","initial_list_status","next_pymnt_d"], axis=1, inplace=True)

In [18]:
df_2.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verified_num,risk_num,issue_num,initial_num,next_pay_num,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,pymnt_plan_n
0,10500.0,0.1719,375.35,3,66000.0,27.24,0.0,0.0,8.0,0.0,...,1,0,3,0,5,1,0,1,1,1
1,25000.0,0.2000,929.09,2,105000.0,20.23,0.0,0.0,17.0,1.0,...,1,0,3,0,5,1,0,1,1,1
2,20000.0,0.2000,529.88,2,56000.0,24.26,0.0,0.0,8.0,0.0,...,1,0,3,0,5,1,0,1,1,1
3,10000.0,0.1640,353.55,3,92000.0,31.44,0.0,1.0,10.0,1.0,...,1,0,3,0,5,1,0,1,1,1
4,22000.0,0.1474,520.39,2,52000.0,18.76,0.0,1.0,14.0,0.0,...,0,0,3,0,5,1,0,1,1,1


## Split the Data into Training and Testing

In [19]:
# Create our features
X = df_2.drop(columns="risk_num")

# Create our target
y = df_2['risk_num']

In [20]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,total_il_high_credit_limit,verified_num,issue_num,initial_num,next_pay_num,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N,pymnt_plan_n
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,5.572240e+04,0.521993,1.726172,0.123879,4.616839,0.860340,0.139660,1.0,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,5.095845e+04,0.499520,0.743862,0.329446,0.486161,0.346637,0.346637,0.0,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.270000e+02,0.000000,1.000000,0.000000,4.000000,0.000000,0.000000,1.0,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,2.288000e+04,0.000000,1.000000,0.000000,4.000000,1.000000,0.000000,1.0,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,4.200000e+04,1.000000,2.000000,0.000000,5.000000,1.000000,0.000000,1.0,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,7.249900e+04,1.000000,2.000000,0.000000,5.000000,1.000000,0.000000,1.0,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.426964e+06,1.000000,3.000000,1.000000,5.000000,1.000000,1.000000,1.0,1.0,1.0


In [21]:
# Check the balance of our target values
y.value_counts()

0    68470
1      347
Name: risk_num, dtype: int64

In [22]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [23]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


In [24]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [25]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [26]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [27]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_rf = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred_rf)

0.7796249964747735

In [28]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_rf)

array([[15476,  1642],
       [   30,    57]])

In [29]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.90      0.66      0.95      0.77      0.61     17118
          1       0.03      0.66      0.90      0.06      0.77      0.58        87

avg / total       0.99      0.90      0.66      0.94      0.77      0.61     17205



In [30]:
# List the features sorted in descending order by feature importance
#? Not sure what this is, sorry!

### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)
y_pred_eec = eec.predict(X_test)


In [ ]:
# Calculated the balanced accuracy score

balanced_accuracy_score(y_test, y_pred_eec)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_eec)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_eec))

### Final Questions

1. Which model had the best balanced accuracy score?

    Easy ensemble classifier.

2. Which model had the best recall score?

    Easy ensemble classifier.

3. Which model had the best geometric mean score?

    Easy ensemble classifier.

4. What are the top three features?

    I think precision, recall, F1.